# 装包以及初始化

In [10]:
%pip install python-dotenv
%pip install langchain
%pip install openai
%pip install tiktoken
%pip install unstructured
%pip install chromadb
%pip install pinecone-client
%pip install ipywidgets
%pip install youtube-transcript-api
%pip install google-search-results
# %pip install pytube

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
  Obtaining dependency information for ipywidgets from https://files.pythonhosted.org/packages/b8/d4/ce436660098b2f456e2b8fdf76d4f33cbc3766c874c4aa2f772c7a5e943f/ipywidgets-8.1.0-py3-none-any.whl.metadata
  Obtaining dependency information for widgetsnbextension~=4.0.7 from https://files.pythonhosted.org/packages/8e/d4/d31b12ac0b87e8cc9fdb6ea1eb6596de405eaaa2f25606aaa755d0eebbc0/widgetsnbextension-4.0.8-py3-none-any.whl.metadata
  Obtaining dependency information for jupyterlab-widgets~=3.0.7 from https://files.pythonhosted.org/pack

In [27]:
import os
from dotenv import load_dotenv

load_dotenv(dotenv_path="./config/.env")
print(os.environ["OPENAI_API_TYPE"])

azure


# 使用 LangChain 完成一次问答

In [2]:
from langchain.chat_models import AzureChatOpenAI
from langchain.schema import HumanMessage

llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo-16k",
    model_name="gpt-35-turbo-16k",
)
msg = HumanMessage(content="怎么评价人工智能")
llm(messages=[msg])

AIMessage(content='人工智能是一种具有潜力和前景的技术，它能够模拟人类的智能和学习能力。以下是对人工智能的评价：\n\n1. 潜力巨大：人工智能有潜力改变我们的生活方式和工作方式。它可以为我们提供更高效、更准确的解决方案，并创造出全新的商业模式和机会。\n\n2. 提高效率：人工智能能够处理大量的数据和信息，并进行快速的分析和推理。这将大大提高工作效率，减少人力成本，并帮助我们更好地解决复杂的问题。\n\n3. 自动化和智能化：人工智能可以实现自动化和智能化，使得机器能够自主地进行决策和执行任务。这将减轻人类的工作负担，使我们能够更专注于创造性的工作和创新。\n\n4. 持续发展和进步：人工智能技术在不断发展和进步，越来越多的领域开始应用人工智能。这将带来更多的创新和突破，进一步推动科学和技术的发展。\n\n5. 需要注意的问题：人工智能也面临一些问题和挑战，例如隐私和数据安全的问题，以及人类对于工作和职业的改变。因此，在推进人工智能的同时，我们还需要关注其社会和伦理影响，并制定相应的政策和规范。\n\n总的来说，人工智能是一项具有巨大潜力和前景的技术，它将对我们的生活和工作产生积极的影响。然而，我们也需要认识到其中存在的问题和挑战，并积极应对和解决。', additional_kwargs={}, example=False)

# 通过 Google 搜索并返回答案

In [7]:
from langchain.agents import load_tools
from langchain.agents import initialize_agent
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import AgentType

# 加载 AzureChatOpenAI 模型
llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo-16k",
    model_name="gpt-35-turbo-16k",
    temperature=0,
    max_tokens=2048,
)

 # 加载 serpapi 工具
tools = load_tools(["serpapi"])
# 工具加载后都需要初始化，verbose 参数为 True，会打印全部的执行详情
agent = initialize_agent(tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# 运行 agent
agent.run("What's the date today? What great events have taken place today in history?")



> Entering new AgentExecutor chain...
I can use the search engine to find the current date and search for historical events that have taken place on this day.
Action: Search
Action Input: "current date"
Observation: ['Current Time (World Clock) and online and printable Calendars for countries worldwide. Find the best time for web meetings (Meeting Planner) or use the Time ...']
Thought:This search result doesn't provide the current date. I should try a different search query.
Action: Search
Action Input: "current date"
Observation: ['Current Time (World Clock) and online and printable Calendars for countries worldwide. Find the best time for web meetings (Meeting Planner) or use the Time ...']
Thought:This search result still doesn't provide the current date. I should try a different search query.
Action: Search
Action Input: "today's date"
Observation: ["Today's Date is Tuesday September 12, 2023. Time zone: California/Mountain View. Change Time zone. Enter Zip Code: September 2023.

'The current date is Tuesday, September 12, 2023. Some historical events that have taken place on this day include the birth of American actress Jennifer Hudson in 1981 and the signing of the Treaty of Portsmouth, ending the Russo-Japanese War, in 1905.'

# 对超长文本进行总结

In [5]:
from langchain.document_loaders import TextLoader
from langchain.prompts import PromptTemplate
from langchain.chains.summarize import load_summarize_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chat_models import AzureChatOpenAI

# 导入文本
loader = TextLoader("./data/novel.txt", encoding="utf8")
# 将文本转成 Document 对象
document = loader.load()
print(f'documents:{len(document)}')

# 初始化文本分割器
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 500,
    chunk_overlap = 0
)

# 切分文本
split_documents = text_splitter.split_documents(document)
print(f'documents:{len(split_documents)}')

# 加载 llm 模型
llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo-16k",
    model_name="gpt-35-turbo-16k",
    max_tokens=1500,
)

# 创建自定义 prompt
question_prompt_str = """
Write a concise summary of the following in Chinese:
------------
{text}
------------
CONCISE SUMMARY:
"""
question_prompt = PromptTemplate(template=question_prompt_str, input_variables=["text"])

refine_prompt_str = """Your job is to produce a final summary.
We have provided an existing summary up to a certain point: This passage describes the layout and atmosphere of a hotel in Lu Town. It mentions the availability of hot water for warming wine and the option to purchase food alongside drinks. The narrator worked as an attendant at the hotel but was eventually assigned the monotonous task of warming wine. The narrator finds the job dull and the customers demanding, except for one customer named Kong Yiji who brings some laughter to the otherwise serious environment. Kong Yiji is the only person who stands while drinking and wears a long gown. He is tall with a pale face and scars on his wrinkled face. His beard is messy and gray. Although his long gown is dirty and torn, it hasn't been repaired or washed in over ten years. According to rumors, Kong Yiji was once educated but never pursued further education and lacks any practical skills. As a result, he became increasingly poor and was on the verge of begging. However, he had a talent for calligraphy and would exchange his written work for food. Unfortunately, he also had a bad habit of drinking too much and being lazy. Within a few days of his arrival, he would steal anything he could find, including books and writing materials. This behavior caused people to stop asking him to write for them. Kong Yiji had no choice but to occasionally resort to theft. However, his behavior in the hotel was better than others, as he never owed money and always paid off his debts promptly. Although he sometimes couldn't pay immediately, he would have it recorded on a slate, but within a month, he would always clear the debt. Kong Yiji drinks half a bowl of wine and his flushed face gradually returns to normal. Other people then ask him if he really knows how to read. Kong Yiji looks at them with a disdainful expression and they continue to mock him for not even being able to become a scholar. This immediately puts Kong Yiji in a state of melancholy and unease, and his face turns gray while he mutters words that no one understands. This causes everyone to burst into laughter, filling the atmosphere inside and outside the hotel with joy. The passage also reveals that the narrator can pretend to laugh along with the customers without being reprimanded by the manager. Kong Yiji often tries to engage in conversation with the customers, but they mock him and question his literacy skills. The narrator dismisses Kong Yiji's attempt to teach him how to write and leaves, causing Kong Yiji to express disappointment.
We have the opportunity to refine the existing summary (only if needed) with some more context below.
------------
{text}
------------
Given the new context, refine the original summary in Chinese.
If the context isn't useful, return the original summary."""
refine_prompt = PromptTemplate(template=refine_prompt_str, input_variables=["text"])

# 创建总结链
chain = load_summarize_chain(llm, chain_type="refine", verbose=True, return_intermediate_steps=True, question_prompt=question_prompt, refine_prompt=refine_prompt)

# 执行总结链，（为了快速演示，只总结前5段）
chain({"input_documents": split_documents[:3]}, return_only_outputs=True)

documents:1
documents:7


> Entering new RefineDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Write a concise summary of the following in Chinese:
------------
魯鎭的酒店的格局，是和別處不同的：都是當街一個曲尺形的大櫃臺，櫃裏面豫備着熱水，可以隨時溫酒。做工的人，傍午傍晚散了工，每每花四文銅錢，買一碗酒，——這是二十多年前的事，現在每碗要漲到十文，——靠櫃外站着，熱熱的喝了休息；倘肯多花一文，便可以買一碟鹽煮筍，或者茴香豆，做下酒物了，如果出到十幾文，那就能買一樣葷菜，但這些顧客，多是短衣幫，大抵沒有這樣闊綽。只有穿長衫的，纔踱進店面隔壁的房子裏，要酒要菜，慢慢地坐喝。

我從十二歲起，便在鎭口的咸亨酒店裏當夥計，掌櫃說，樣子太傻，怕侍候不了長衫主顧，就在外面做點事罷。外面的短衣主顧，雖然容易說話，但嘮嘮叨叨纏夾不清的也很不少。他們往往要親眼看着黃酒從罎子裏舀出，看過壺子底裏有水沒有，又親看將壺子放在熱水裏，然後放心：在這嚴重監督下，羼水也很爲難。所以過了幾天，掌櫃又說我幹不了這事。幸虧薦頭的情面大，辭退不得，便改爲專管溫酒的一種無聊職務了。

我從此便整天的站在櫃臺裏，專管我的職務。雖然沒有什麼失職，但總覺得有些單調，有些無聊。掌櫃是一副凶臉孔，主顧也沒有好聲氣，教人活潑不得；只有孔乙己到店，纔可以笑幾聲，所以至今還記得。
------------
CONCISE SUMMARY:


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
Your job is to produce a final summary.
We have provided an existing summary up to a certain point: This passage describes the layout and atmosphere of a hotel in Lu Town. It mentions the availability

{'intermediate_steps': ['这篇文章描述了鲁镇一家酒店的情景。酒店的格局独特，大厅设有一个弯曲的柜台，柜台里备有热水，可以随时温酒。文章主要描述了作者在酒店工作的经历，他在酒店当夥计，负责温酒的任务。工作虽然没有什么失职，但感觉单调而乏味。只有一个叫孔乙己的顾客来店，才能让作者开心笑几声。',
  '这段文字描述了鲁镇一家旅馆的布局和氛围。它提到了有热水供应可以用来温酒，以及购买食物和酒水的选择。叙述者曾在旅馆担任服务员，但最终被派去做单调乏味的温酒工作。叙述者觉得这份工作枯燥无聊，顾客们要求很高，除了一个名叫孔乙己的顾客之外。孔乙己是唯一一个站着喝酒、穿着长衫的人。他个子高大，脸色苍白，皱纹上有伤痕，胡子凌乱而花白。尽管他的长衫又脏又破，十多年来既没有修补也没有洗涤，但他从未追求过进修教育，缺乏实用技能。结果，他日益贫困，濒临乞讨。然而，他有书法才华，会用自己的书法作品换取食物。不幸的是，他喝酒过量且懒惰，几天来他会偷走任何他能找到的东西，包括书籍和写字工具。这种行为导致人们不再向他请教写字，孔乙己别无选择，只能偶尔诈骗。然而，与其他人相比，他在旅馆的行为更好，从不欠债，总是及时偿还。尽管有时付不起，他会把欠款记录在石板上，但一个月内总会清偿债务。孔乙己喝了半碗酒，脸色渐渐恢复正常。其他人便问他是否真的会读书。孔乙己用轻蔑的表情看着他们，他们继续嘲笑他甚至连一个读书人都做不成。这立刻让孔乙己陷入忧郁和不安的状态，他的脸变灰了，嘟囔着别人听不懂的话。这让大家都笑了起来，旅馆里外充满了欢声笑语。这段文字还透露了叙述者可以假装与顾客一起笑而不受经理责备。孔乙己经常试图与顾客交流，但他们嘲笑他，质疑他的读写能力。叙述者对孔乙己试图教他写字的尝试不屑一顾，并离开，引起孔乙己的失望。',
  '这段描写了吕镇一家酒店的布局和氛围。它提到了为暖酒提供热水的便利以及可以购买食物和饮料的选择。叙述者曾在酒店工作，但最终被安排了单调乏味的任务——暖酒。叙述者觉得这份工作枯燥无味，顾客也很苛刻，除了一个名叫孔乙己的顾客，他给这个本来严肃的环境带来了一些欢笑。孔乙己是唯一一位站着喝酒、穿着长袍的人。他身材高大，面色苍白，满脸皱纹和伤疤。他的胡须凌乱而灰色。虽然他的长袍又脏又破，但已经十多年没有修补或洗过了。根据传闻，孔乙己曾受过教育，但从未继续深造，也没有任何实用技

# 构建本地知识库问答机器人

In [6]:
from langchain.chat_models import AzureChatOpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import CharacterTextSplitter
from langchain import VectorDBQA
from langchain.document_loaders import DirectoryLoader
from langchain.chains import RetrievalQA

# 加载文件夹中的所有txt类型的文件
loader = DirectoryLoader('./data', glob='**/*.txt')
# 将数据转成 document 对象，每个文件会作为一个 document
documents = loader.load()

# 初始化加载器
text_splitter = CharacterTextSplitter(chunk_size=100, chunk_overlap=0)
# 切割加载的 document
split_documents = text_splitter.split_documents(documents)

# 初始化 openai 的 embeddings 对象
embeddings = OpenAIEmbeddings(
    deployment_name="text-embedding-ada-002",
    model_name="text-embedding-ada-002",
    chunk_size=1,
)
# 将 document 通过 openai 的 embeddings 对象计算 embedding向量信息并临时存入 Chroma 向量数据库，用于后续匹配查询
docsearch = Chroma.from_documents(split_documents, embeddings)

# 加载 llm 模型
llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo-16k",
    model_name="gpt-35-turbo-16k",
    max_tokens=1500,
)
# 创建问答对象
qa = VectorDBQA.from_chain_type(llm=llm, chain_type="stuff", vectorstore=docsearch, return_source_documents=True)
# 进行问答
result = qa({"query": "孔乙己是谁？人们对他的评价是什么？"})
print(result)

[nltk_data] Downloading package punkt to /Users/jiyu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/jiyu/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
Created a chunk of size 208, which is longer than the specified 100
Created a chunk of size 188, which is longer than the specified 100
Created a chunk of size 398, which is longer than the specified 100
Created a chunk of size 213, which is longer than the specified 100
Created a chunk of size 160, which is longer than the specified 100
Created a chunk of size 380, which is longer than the specified 100
Created a chunk of size 142, which is longer than the specified 100
Created a chunk of size 238, which is longer than the specified 100
Created a chunk of size 450, which is longer than the specified 100
/Users/jiyu/anaconda3/envs/learn/lib/python3.9/site-packages/langchain/embeddings/openai.py:214: UserWarning:

{'query': '孔乙己是谁？人们对他的评价是什么？', 'result': '孔乙己是一个身材高大，脸色苍白，鬍子花白的人。他穿着破旧的长衫，看起来很脏。他说话时经常使用一些让人难以理解的词语。因为他姓孔，所以人们给他取了一个绰号叫孔乙己。\n\n当孔乙己进入酒店时，所有喝酒的人都会笑着看他，有人会调侃他又添上新伤疤了。他不回答，只对柜台说要两碗酒和一碟茴香豆。他们故意高声嚷道，说他一定又偷了人家的东西。孔乙己会生气地回答，说他怎么会冤枉他，说他偷书不能算偷，因为读书人的事情怎么能算偷呢？他说的一些不容易理解的话引来了大家的笑声。\n\n孔乙己能给人带来快乐，但没有他，大家也会过得一样。当他喝了半碗酒后，脸色恢复了正常，别人会问他是否真的认识字，他会不屑地回答。他们会接着说，他怎么连半个秀才都考不到。孔乙己会显得颓废不安，并且会说一些不容易理解的话，引来大家的笑声。\n\n据人们背地里讲，孔乙己曾经读过书，但最终没有进学，也不会谋生，导致越来越穷，差点要讨饭了。幸好他写字很好，能替别人抄写书籍来换一碗饭吃。不幸的是，他有个坏毛病，就是爱喝酒又懒得做事。坐在这里没几天，他就会把人和书籍纸张笔硯一起偷走。多次如此，找他抄写书籍的人也没了。孔乙己无可奈何，只能偶尔做些偷窃的事。但在我们店里，他的品行比其他人都好，从不拖欠。虽然偶尔没有现钱，暂时记在粉板上，但不到一个月，肯定会还清，从粉板上擦去孔乙己的名字。', 'source_documents': [Document(page_content='孔乙己是站着喝酒而穿長衫的唯一的人。他身材很高大；青白臉色，皺紋間時常夾些傷痕；一部亂蓬蓬的花白的鬍子。穿的雖然是長衫，可是又髒又破，似乎十多年沒有補，也沒有洗。他對人說話，總是滿口之乎者也，教人半懂不懂的。因爲他姓孔，別人便從描紅紙上的「上大人孔乙己」這半懂不懂的話裏，替他取下一個綽號，叫作孔乙己。孔乙己一到店，所有喝酒的人便都看着他笑，有的叫道，「孔乙己，你臉上又添上新傷疤了！」他不回答，對櫃裏說，「溫兩碗酒，要一碟茴香豆。」便排出九文大錢。他們又故意的高聲嚷道，「你一定又偷了人家的東西了！」孔乙己睜大眼睛說，「你怎麼這樣憑空汚人清白……」「什麼清白？我前天親眼見你偷了何家的書，吊着打。」孔乙己便漲紅了臉，額上的青筋條條綻出，爭辯道，「竊書不能算偷……竊書！……讀書人

# 构建向量索引数据库

In [12]:
import os;
from langchain.text_splitter import CharacterTextSplitter
from langchain.document_loaders import DirectoryLoader
from langchain.vectorstores import Chroma, Pinecone
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.chat_models import AzureChatOpenAI
from langchain.chains.question_answering import load_qa_chain

import pinecone

# 初始化 pinecone
pinecone.init(
  api_key=os.environ["PINECONE_API_KEY"],
  environment=os.environ["PINECONE_API_KEY"],
)

loader = DirectoryLoader('./data', glob='**/*.txt')
# 将数据转成 document 对象，每个文件会作为一个 document
documents = loader.load()

# 初始化加载器
text_splitter = CharacterTextSplitter(chunk_size=500, chunk_overlap=0)
# 切割加载的 document
split_docs = text_splitter.split_documents(documents)

# 初始化 openai 的 embeddings 对象
embeddings = OpenAIEmbeddings(
    deployment_name="text-embedding-ada-002",
    model_name="text-embedding-ada-002",
    chunk_size=1,
)

index_name="novel"
# 持久化数据
# docsearch = Pinecone.from_texts([t.page_content for t in split_docs], embeddings, index_name=index_name)

# 加载数据
docsearch = Pinecone.from_existing_index(index_name,embeddings)

query = "孔乙己是谁？"
docs = docsearch.similarity_search(query, include_metadata=True)

llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo-16k",
    model_name="gpt-35-turbo-16k",
    max_tokens=1500,
    temperature=0,
)
chain = load_qa_chain(llm, chain_type="stuff", verbose=True)
chain.run(input_documents=docs, question=query)

MaxRetryError: HTTPSConnectionPool(host='controller.fed60dbd-6101-47c8-99d6-ab5cc1f62acf.pinecone.io', port=443): Max retries exceeded with url: /databases (Caused by NameResolutionError("<urllib3.connection.HTTPSConnection object at 0x7fad49ede460>: Failed to resolve 'controller.fed60dbd-6101-47c8-99d6-ab5cc1f62acf.pinecone.io' ([Errno 8] nodename nor servname provided, or not known)"))

# 使用GPT3.5模型构建油管频道问答机器人


In [ ]:
import os

from langchain.document_loaders import YoutubeLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import ChatVectorDBChain, ConversationalRetrievalChain

from langchain.chat_models import AzureChatOpenAI
from langchain.prompts.chat import (
  ChatPromptTemplate,
  SystemMessagePromptTemplate,
  HumanMessagePromptTemplate
)

# 加载 youtube 频道
loader = YoutubeLoader.from_youtube_url('https://www.youtube.com/watch?v=Dj60HHy-Kqk')
# 将数据转成 document
documents = loader.load()

# 初始化文本分割器
text_splitter = RecursiveCharacterTextSplitter(
  chunk_size=1000,
  chunk_overlap=20
)

# 分割 youtube documents
split_documents = text_splitter.split_documents(documents)
print(split_documents[:1])

# 初始化 openai embeddings
embeddings = OpenAIEmbeddings(
    deployment_name="text-embedding-ada-002",
    model_name="text-embedding-ada-002",
    chunk_size=1,
)

# 将数据存入向量存储
vector_store = Chroma.from_documents(split_documents, embeddings)
# 通过向量存储初始化检索器
retriever = vector_store.as_retriever()

system_template = """
Use the following context to answer the user's question.
If you don't know the answer, just say that you don't know, don't try to make up an answer. And answer in English.
-----------
{context}
-----------
{chat_history}
"""

# 构建初始 messages 列表，这里可以理解为是 openai 传入的 messages 参数
messages = [
  SystemMessagePromptTemplate.from_template(system_template),
  HumanMessagePromptTemplate.from_template('{question}')
]

# 初始化 prompt 对象
prompt = ChatPromptTemplate.from_messages(messages)


# 初始化问答链
llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo-16k",
    model_name="gpt-35-turbo-16k",
    temperature=0.1,
    max_tokens=2048,
)
chain = ConversationalRetrievalChain.from_llm(
    llm,
    retriever,
    combine_docs_chain_kwargs={
        "prompt": prompt,
    },
    verbose=True,
)


chat_history = []
while True:
  question = input('问题：')
  # 开始发送问题 chat_history 为必须参数,用于存储对话历史
  result = chain({'question': question, 'chat_history': chat_history})
  chat_history.append((question, result['answer']))
  print(result['answer'])

[Document(page_content="[Applause] thanks Tim and hi everyone I'm excited to get to show you some of the new experimental features we've been working on for Unreal Engine 5.2 let's take a look [Music] all right so last year we added several new features to the engine to support foliage rendering and the fortnite team uses features to ship battle royale chapter 4. at the same time Jacob over there and the team at qixel were experimenting with what's possible for photoreal foliage environments as well as testing out the latest functionality that we've been building for Unreal Engine so Jacob's here with us today in the unreal editor let's explore the environment and what better way to do that than off-roading and what better way to off-road than an arivian r1t now rivian uses unreal to power their instrument cluster including 3D visualization of their vehicles so we worked with them to bring the r1t to life in this experience let's head on out Jacob sure thing on my way all right so we'r

# 用 OpenAI 连接万种工具

In [7]:
from langchain.chat_models import AzureChatOpenAI
from langchain.agents import initialize_agent
from langchain.agents import AgentType
from langchain.agents.agent_toolkits import ZapierToolkit
from langchain.utilities.zapier import ZapierNLAWrapper

llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo-16k",
    model_name="gpt-35-turbo-16k",
    temperature=.3,
)

zapier = ZapierNLAWrapper()
toolkit = ZapierToolkit.from_zapier_nla_wrapper(zapier)
agent = initialize_agent(toolkit.get_tools(), llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True)

# 我们可以通过打印的方式看到我们都在 Zapier 里面配置了哪些可以用的工具
for tool in toolkit.get_tools():
  print (tool.name)
  print (tool.description)
  print ("\n\n")

Gmail: Send Email
A wrapper around Zapier NLA actions. The input to this tool is a natural language instruction, for example "get the latest email from my bank" or "send a slack message to the #general channel". Each tool will have params associated with it that are specified as a list. You MUST take into account the params when creating the instruction. For example, if the params are ['Message_Text', 'Channel'], your instruction should be something like 'send a slack message to the #general channel with the text hello world'. Another example: if the params are ['Calendar', 'Search_Term'], your instruction should be something like 'find the meeting in my personal calendar at 3pm'. Do not make up params, they will be explicitly specified in the tool description. If you do not have enough information to fill in the params, just say 'not enough information provided in the instruction, missing <param>'. If you get a none or null response, STOP EXECUTION, do not try to another tool!This too

In [8]:
agent.run('请发送一封邮件给"jiyu.shao@gmail.com"，内容是"Hello from LangChain"。')



> Entering new AgentExecutor chain...
I need to send an email to "jiyu.shao@gmail.com" with the subject "Hello from LangChain" and the body of the email.
Action: Gmail: Send Email
Action Input: 
- Subject: "Hello from LangChain"
- Body: "Hello from LangChain"
- To: "jiyu.shao@gmail.com"
Observation: {"id": "18a841d336f8e579", "threadId": "18a841d336f8e579", "labelIds": ["UNREAD", "SENT", "INBOX"]}
Thought:The email has been sent successfully.
Final Answer: The email has been sent to "jiyu.shao@gmail.com" with the subject "Hello from LangChain" and the body "Hello from LangChain".

> Finished chain.


'The email has been sent to "jiyu.shao@gmail.com" with the subject "Hello from LangChain" and the body "Hello from LangChain".'

# 一些有意思的小Tip

## 执行多个chain

In [13]:
from langchain.chat_models import AzureChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.chains import SimpleSequentialChain

# location 链
llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo-16k",
    model_name="gpt-35-turbo-16k",
    temperature=1,
)

template = """Your job is to come up with a classic dish from the area that the users suggests.
% USER LOCATION
{user_location}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_location"], template=template)
location_chain = LLMChain(llm=llm, prompt=prompt_template)

# meal 链
template = """Given a meal, give a short and simple recipe on how to make that dish at home.
% MEAL
{user_meal}

YOUR RESPONSE:
"""
prompt_template = PromptTemplate(input_variables=["user_meal"], template=template)
meal_chain = LLMChain(llm=llm, prompt=prompt_template)

# 通过 SimpleSequentialChain 串联起来，第一个答案会被替换第二个中的user_meal，然后再进行询问
overall_chain = SimpleSequentialChain(chains=[location_chain, meal_chain], verbose=True)
review = overall_chain.run("China")



> Entering new SimpleSequentialChain chain...
One classic dish from China is Peking Duck. Peking Duck is a dish that originated in Beijing and is considered one of the most famous dishes in Chinese cuisine. It is made by roasting a whole duck until the skin is crispy and golden while the meat remains tender and juicy. The duck is traditionally served with thin pancakes, spring onions, cucumber, and hoisin sauce. This dish is not only delicious but also showcases the skill and artistry of Chinese cuisine.
Ingredients:
- 1 whole duck
- Salt
- 1 tablespoon honey
- 1 tablespoon soy sauce
- 1 tablespoon rice wine or dry sherry
- Thin pancakes or tortillas
- Spring onions
- Cucumber
- Hoisin sauce

Instructions:
1. Preheat your oven to 350°F (175°C).
2. Clean the duck thoroughly and pat it dry.
3. Rub the inside and outside of the duck with salt to season.
4. Place the duck on a rack in a roasting pan, breast side up.
5. Roast the duck for approximately 3 hours, or until the skin is crispy

## 结构化输出

In [14]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema
from langchain.prompts import PromptTemplate
from langchain.chat_models import AzureChatOpenAI

llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo-16k",
    model_name="gpt-35-turbo-16k",
    temperature=1,
)

# 告诉他我们生成的内容需要哪些字段，每个字段类型式啥
response_schemas = [
    ResponseSchema(name="bad_string", description="This a poorly formatted user input string"),
    ResponseSchema(name="good_string", description="This is your response, a reformatted response")
]

# 初始化解析器
output_parser = StructuredOutputParser.from_response_schemas(response_schemas)

# 生成的格式提示符
# {
#	"bad_string": string  // This a poorly formatted user input string
#	"good_string": string  // This is your response, a reformatted response
#}
format_instructions = output_parser.get_format_instructions()

template = """
You will be given a poorly formatted string from a user.
Reformat it and make sure all the words are spelled correctly

{format_instructions}

% USER INPUT:
{user_input}

YOUR RESPONSE:
"""

# 讲我们的格式描述嵌入到 prompt 中去，告诉 llm 我们需要他输出什么样格式的内容
prompt = PromptTemplate(
    input_variables=["user_input"],
    partial_variables={"format_instructions": format_instructions},
    template=template
)

chain = LLMChain(llm=llm, prompt=prompt)
llm_output = chain.run("welcom to china!")

# 使用解析器进行解析生成的内容
output_parser.parse(llm_output)

{'bad_string': 'welcom to china!', 'good_string': 'Welcome to China!'}

## 爬取网页并输出JSON数据

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMRequestsChain, LLMChain

llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo-16k",
    model_name="gpt-35-turbo-16k",
    temperature=0,
)

template = """在 >>> 和 <<< 之间是网页的返回的HTML内容。
网页是新浪财经A股上市公司的公司简介。
请抽取参数请求的信息。

>>> {requests_result} <<<
请使用如下的JSON格式返回数据
{{
  "company_name":"a",
  "company_english_name":"b",
  "issue_price":"c",
  "date_of_establishment":"d",
  "registered_capital":"e",
  "office_address":"f",
  "Company_profile":"g"

}}
Extracted:"""

prompt = PromptTemplate(
    input_variables=["requests_result"],
    template=template
)

chain = LLMRequestsChain(llm_chain=LLMChain(llm=llm, prompt=prompt))
inputs = {
  "url": "https://vip.stock.finance.sina.com.cn/corp/go.php/vCI_CorpInfo/stockid/600519.phtml"
}

response = chain(inputs)
print(response['output'])

{
  "company_name":"贵州茅台酒股份有限公司",
  "company_english_name":"Kweichow Moutai Co.,Ltd.",
  "issue_price":"31.39",
  "date_of_establishment":"1999-11-20",
  "registered_capital":"125620万元(CNY)",
  "office_address":"贵州省仁怀市茅台镇",
  "Company_profile":"公司是根据贵州省人民政府黔府函〔1999〕291号文,由中国贵州茅台酒厂有限责任公司作为主发起人,联合贵州茅台酒厂技术开发公司、贵州省轻纺集体工业联社、深圳清华大学研究院、中国食品发酵工业研究院、北京市糖业烟酒公司、江苏省糖烟酒总公司、上海捷强烟草糖酒(集团)有限公司于1999年11月20日共同发起设立的股份有限公司。经中国证监会证监发行字[2001]41号文核准并按照财政部企[2001]56号文件的批复,公司于2001年7月31日在上海证券交易所公开发行7,150万(其中,国有股存量发行650万股)A股股票。",
}


## 自定义工具

In [32]:
from langchain.agents import initialize_agent, Tool
from langchain.agents import AgentType
from langchain.schema import SystemMessage
from langchain.tools import BaseTool
from langchain.llms import OpenAI
from langchain import LLMMathChain, SerpAPIWrapper

llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo-16k",
    model_name="gpt-35-turbo-16k",
    temperature=0,
)

# 初始化搜索链和计算链
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)

# 生成一个功能列表，指明这个 agent 里面都有哪些可用工具
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

# 初始化 agent
system_message = SystemMessage(content="Your role is to answer the user's questions in a human-like manner. Please respond in Chinese")
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    system_message=system_message,
    agent_kwargs={
        "system_message": system_message
    },
    verbose=True
)

# 执行 agent
agent.run("今天日期是什么？今年国庆啥时间？放几天假？")
agent.run("12 的 12 次方是多少？")

/Users/jiyu/anaconda3/envs/learn/lib/python3.9/site-packages/langchain/chains/llm_math/base.py:50: UserWarning: Directly instantiating an LLMMathChain with an llm is deprecated. Please instantiate with llm_chain argument or using the from_llm class method.
  warnings.warn(




> Entering new AgentExecutor chain...
I need to find out today's date and the date of the National Day holiday this year, as well as the duration of the holiday.
Action: Search
Action Input: "today's date", "National Day date this year", "duration of National Day holiday"
Observation: ['Thursday, September 14th.']
Thought:I have found today's date.
Action: Search
Action Input: "National Day date this year"
Observation: ['Thursday, September 14th.']
Thought:I have found the date of the National Day holiday this year.
Action: Search
Action Input: "duration of National Day holiday"
Observation: ['NATIONAL CREAM-FILLED DONUT DAY – September 14.']
Thought:I couldn't find the duration of the National Day holiday.
Action: Search
Action Input: "duration of National Day holiday in China"
Observation: one week
Thought:I now know the final answer
Final Answer: Today's date is September 14th. The date of the National Day holiday this year is also September 14th. The duration of the National Day 

'12 raised to the power of 12 is 8,916,100,448,256.'

## 使用Memory实现一个带记忆的对话机器人

In [33]:
from langchain.memory import ChatMessageHistory
from langchain.chat_models import ChatOpenAI

llm = AzureChatOpenAI(
    deployment_name="gpt-35-turbo-16k",
    model_name="gpt-35-turbo-16k",
    temperature=1,
)

# 初始化 MessageHistory 对象
history = ChatMessageHistory()

# 给 MessageHistory 对象添加对话内容
history.add_ai_message("你好！")
history.add_user_message("中国的首都是哪里？")

# 执行对话
ai_response = llm(history.messages)
print(ai_response)

content='中国的首都是北京。' additional_kwargs={} example=False
